In [1]:
#This is a dummie file for a git hub push test
import numpy as np
import pandas as pd
import random
import matplotlib.pyplot as plt
import seaborn as sb
import statsmodels as smodel
import sklearn
from sklearn import datasets, linear_model, metrics
import statsmodels.formula.api as stmf
import os


# NY DATA
nyDataFrame = pd.read_csv('AB_NYC_2019.csv')
nyDataFrame['coordinates'] = nyDataFrame['latitude'].astype(str) + ', ' + nyDataFrame['longitude'].astype(str)
del nyDataFrame['latitude']
del nyDataFrame['longitude']
nyDataFrame


,id,name,host_id,host_name,neighbourhood_group,neighbourhood,room_type,price,minimum_nights,number_of_reviews,last_review,reviews_per_month,calculated_host_listings_count,availability_365,coordinates
0,2539,Clean & quiet apt home by the park,2787,John,Brooklyn,Kensington,Private room,149,1,9,2018-10-19,0.21,6,365,"40.647490000000005, -73.97237"
1,2595,Skylit Midtown Castle,2845,Jennifer,Manhattan,Midtown,Entire home/apt,225,1,45,2019-05-21,0.38,2,355,"40.75362, -73.98376999999999"
2,3647,THE VILLAGE OF HARLEM....NEW YORK !,4632,Elisabeth,Manhattan,Harlem,Private room,150,3,0,NaN,NaN,1,365,"40.809020000000004, -73.9419"
3,3831,Cozy Entire Floor of Brownstone,4869,LisaRoxanne,Brooklyn,Clinton Hill,Entire home/apt,89,1,270,2019-07-05,4.64,1,194,"40.685140000000004, -73.95976"
4,5022,Entire Apt: Spacious Studio/Loft by central park,7192,Laura,Manhattan,East Harlem,Entire home/apt,80,10,9,2018-11-19,0.10,1,0,"40.79851, -73.94399"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
48890,36484665,Charming one bedroom - newly renovated rowhouse,8232441,Sabrina,Brooklyn,Bedford-Stuyvesant,Private room,70,2,0,NaN,NaN,2,9,"40.67853, -73.94995"
48891,36485057,Affordable room in Bushwick/East Williamsburg,6570630,Marisol,Brooklyn,Bushwick,Private room,40,4,0,NaN,NaN,2,36,"40.701840000000004, -73.93316999999999"
48892,36485431,Sunny Studio at Historical Neighborhood,23492952,Ilgar & Aysel,Manhattan,Harlem,Entire home/apt,115,10,0,NaN,NaN,1,27,"40.81475, -73.94866999999999"
48893,36485609,43rd St. Time Square-cozy single bed,30985759,Taz,Manhattan,Hell's Kitchen,Shared room,55,1,0,NaN,NaN,6,2,"40.757509999999996, -73.99112"


In [2]:

# USA DATA
usaDataFrame = pd.read_csv('AB_US_2020.csv')
del usaDataFrame['neighbourhood_group']
usaDataFrame['coordinates'] = usaDataFrame['latitude'].astype(str) + ', ' + usaDataFrame['longitude'].astype(str)
del usaDataFrame['latitude']
del usaDataFrame['longitude']
usaDataFrame


/opt/conda/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3145: DtypeWarning: Columns (4) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


,id,name,host_id,host_name,neighbourhood,room_type,price,minimum_nights,number_of_reviews,last_review,reviews_per_month,calculated_host_listings_count,availability_365,city,coordinates
0,38585,Charming Victorian home - twin beds + breakfast,165529,Evelyne,28804,Private room,60,1,138,16/02/20,1.14,1,0,Asheville,"35.65146, -82.62791999999999"
1,80905,French Chic Loft,427027,Celeste,28801,Entire home/apt,470,1,114,07/09/20,1.03,11,288,Asheville,"35.59779, -82.5554"
2,108061,Walk to stores/parks/downtown. Fenced yard/Pet...,320564,Lisa,28801,Entire home/apt,75,30,89,30/11/19,0.81,2,298,Asheville,"35.6067, -82.55563000000001"
3,155305,Cottage! BonPaul + Sharky's Hostel,746673,BonPaul,28806,Entire home/apt,90,1,267,22/09/20,2.39,5,0,Asheville,"35.57864, -82.59578"
4,160594,Historic Grove Park,769252,Elizabeth,28801,Private room,125,30,58,19/10/15,0.52,1,0,Asheville,"35.61442, -82.54127"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
226025,45506143,"DC Hidden In Plain ""Site""",25973146,Marci,"Downtown, Chinatown, Penn Quarters, Mount Vern...",Entire home/apt,104,1,0,NaN,NaN,2,99,Washington D.C.,"38.90388, -77.02973"
226026,45511428,DC 3 BR w/ screen porch 3 blck to metro w/ par...,231133074,Thomas,"Brookland, Brentwood, Langdon",Entire home/apt,151,2,0,NaN,NaN,1,300,Washington D.C.,"38.92082, -76.99098000000001"
226027,45514685,Charming Penthouse Apt w/ Rooftop Terrace in L...,33758935,Bassem,"Shaw, Logan Circle",Entire home/apt,240,2,0,NaN,NaN,1,173,Washington D.C.,"38.91117, -77.03354"
226028,45516412,Adams Morgan/Nat'l Zoo 1 BR Apt #32,23193071,Michael,"Kalorama Heights, Adams Morgan, Lanier Heights",Entire home/apt,60,21,0,NaN,NaN,5,362,Washington D.C.,"38.926629999999996, -77.04436"


In [3]:
# SEATTLE DATA
seattleDataFrame = pd.read_csv('seattle_01.csv')
del seattleDataFrame['rate_type']
seattleDataFrame['coordinates'] = seattleDataFrame['latitude'].astype(str) + ', ' + seattleDataFrame['longitude'].astype(str)
del seattleDataFrame['latitude']
del seattleDataFrame['longitude']
seattleDataFrame

,Unnamed: 0,room_id,host_id,room_type,address,reviews,overall_satisfaction,accommodates,bedrooms,bathrooms,price,last_modified,location,name,currency,coordinates
0,0,2318,2536,Entire home/apt,"Seattle, WA, United States",21,5.0,8,4.0,2.5,250,2018-12-20 03:46:14.698548,0101000020E6100000D449B6BA9C925EC0416326512FCE...,"Casa Madrona - Urban Oasis, 1 block from the P...",USD,"47.610819, -122.290816"
1,1,3335,4193,Entire home/apt,"Seattle, WA, United States",1,NaN,4,2.0,1.0,100,2018-12-20 04:08:45.277643,0101000020E61000006FBBD05CA7915EC04DF564FED1C3...,Sweet Seattle Urban Homestead 2 Bdr,USD,"47.529846, -122.27584"
2,2,4291,35749,Private room,"Seattle, WA, United States",63,4.5,2,1.0,1.0,82,2018-12-20 03:04:19.861014,0101000020E6100000BDAB1E300F945EC0FB93F8DC09D8...,Sunrise in Seattle Master Suite,USD,"47.687801, -122.313427"
3,3,5682,8993,Entire home/apt,"Seattle, WA, United States",462,5.0,2,0.0,1.0,49,2018-12-20 04:11:25.482975,0101000020E6100000FCC7427408975EC009E1D1C611C3...,"Cozy Studio, min. to downtown -WiFi",USD,"47.52398, -122.359891"
4,4,6606,14942,Entire home/apt,"Seattle, WA, United States",134,4.5,2,1.0,1.0,90,2018-12-20 03:12:38.391711,0101000020E6100000D47D00529B955EC07782FDD7B9D3...,"Fab, private seattle urban cottage!",USD,"47.65410900000001, -122.337605"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7571,7970,30976187,231435350,Entire home/apt,"Seattle, WA, United States",0,NaN,4,1.0,1.0,78,2018-12-20 03:56:11.441877,0101000020E610000001F6D1A92B955EC0EFAA07CC43CE...,Lovely Downtown Condo WS 97 (Includes Parking),USD,"47.611444, -122.33079"
7572,7971,30982022,215426444,Private room,"Seattle, WA, United States",0,NaN,3,1.0,1.0,34,2018-12-20 03:42:51.04026,0101000020E610000038656EBE11935EC006F2ECF2ADC9...,2 Beds&Bath Free Parking& LR STA Airport DT UW...,USD,"47.57562100000001, -122.297958"
7573,7972,30994069,230457966,Entire home/apt,"Seattle, WA, United States",0,NaN,2,1.0,1.0,80,2018-12-20 03:19:17.810312,0101000020E610000082397AFCDE945EC00DA9A27895CF...,Vintage Private Courtyard Apartment,USD,"47.621749, -122.32611"
7574,7973,30998202,48005494,Entire home/apt,"Washington, WA, United States",0,NaN,4,2.0,2.0,129,2018-12-20 03:26:52.53791,0101000020E610000014CB2DAD86955EC0DCD8EC48F5CF...,2BR Apartment,USD,"47.624673, -122.336345"
